In [3]:
import pandas as pd
import sqlite3
conn = sqlite3.connect("chinook.db")
cursor = conn.cursor()
q = "SELECT * FROM sqlite_master WHERE type='table';"
cursor.execute(q).fetchall()b

[('table',
  'album',
  'album',
  2,
  'CREATE TABLE [album]\n(\n    [album_id] INTEGER PRIMARY KEY NOT NULL,\n    [title] NVARCHAR(160)  NOT NULL,\n    [artist_id] INTEGER  NOT NULL,\n    FOREIGN KEY ([artist_id]) REFERENCES [artist] ([artist_id]) \n\t\tON DELETE NO ACTION ON UPDATE NO ACTION\n)'),
 ('table',
  'artist',
  'artist',
  3,
  'CREATE TABLE [artist]\n(\n    [artist_id] INTEGER PRIMARY KEY NOT NULL,\n    [name] NVARCHAR(120)\n)'),
 ('table',
  'customer',
  'customer',
  4,
  'CREATE TABLE [customer]\n(\n    [customer_id] INTEGER PRIMARY KEY NOT NULL,\n    [first_name] NVARCHAR(40)  NOT NULL,\n    [last_name] NVARCHAR(20)  NOT NULL,\n    [company] NVARCHAR(80),\n    [address] NVARCHAR(70),\n    [city] NVARCHAR(40),\n    [state] NVARCHAR(40),\n    [country] NVARCHAR(40),\n    [postal_code] NVARCHAR(10),\n    [phone] NVARCHAR(24),\n    [fax] NVARCHAR(24),\n    [email] NVARCHAR(60)  NOT NULL,\n    [support_rep_id] INTEGER,\n    FOREIGN KEY ([support_rep_id]) REFERENCES [empl

In [4]:
pd.read_sql_query(q, conn)

,type,name,tbl_name,rootpage,sql
0,table,album,album,2,CREATE TABLE [album]\n(\n [album_id] INTEGE...
1,table,artist,artist,3,CREATE TABLE [artist]\n(\n [artist_id] INTE...
2,table,customer,customer,4,CREATE TABLE [customer]\n(\n [customer_id] ...
3,table,employee,employee,5,CREATE TABLE [employee]\n(\n [employee_id] ...
4,table,genre,genre,6,CREATE TABLE [genre]\n(\n [genre_id] INTEGE...
5,table,invoice,invoice,7,CREATE TABLE [invoice]\n(\n [invoice_id] IN...
6,table,invoice_line,invoice_line,8,CREATE TABLE [invoice_line]\n(\n [invoice_l...
7,table,media_type,media_type,9,CREATE TABLE [media_type]\n(\n [media_type_...
8,table,playlist,playlist,10,CREATE TABLE [playlist]\n(\n [playlist_id] ...
9,table,playlist_track,playlist_track,11,CREATE TABLE [playlist_track]\n(\n [playlis...


# summary data for every playlist in the Chinook database

In [5]:
playlist_q = '''
            WITH playlist_info AS
                (
                 SELECT
                     p.playlist_id,
                     p.name playlist_name,
                     t.name track_name,
                     (t.milliseconds / 1000) length_seconds  
                 FROM playlist p
                 LEFT JOIN playlist_track pt ON pt.playlist_id = p.playlist_id
                 LEFT JOIN track t ON t.track_id = pt.track_id
                )
            SELECT
                playlist_id,
                playlist_name,
                COUNT(track_name) number_of_tracks,
                SUM(length_seconds) length_seconds
            FROM playlist_info
            GROUP BY 1,2 ORDER BY 1;
'''
pd.read_sql_query(playlist_q, conn)

,playlist_id,playlist_name,number_of_tracks,length_seconds
0,1,Music,3290,876049.0
1,2,Movies,0,NaN
2,3,TV Shows,213,500987.0
3,4,Audiobooks,0,NaN
4,5,90’s Music,1477,397970.0
5,6,Audiobooks,0,NaN
6,7,Movies,0,NaN
7,8,Music,3290,876049.0
8,9,Music Videos,1,294.0
9,10,TV Shows,213,500987.0


# named subqueries

In [10]:
india = '''
            WITH
                india AS
                      (
                      SELECT * FROM customer
                      WHERE country = "India"
                      ),
                sum_total AS
                      (
                      SELECT 
                          customer_id,
                          SUM(total) total
                      FROM invoice
                      GROUP BY customer_id
                      )
            SELECT
                i.first_name || " " || i.last_name customer_name,
                st.total total_purchases
            FROM india i
            INNER JOIN sum_total st ON st.customer_id = i.customer_id
            ORDER BY 1;
'''
pd.read_sql_query(india, conn)

,customer_name,total_purchases
0,Manoj Pareek,111.87
1,Puja Srivastava,71.28


# Each country's best customer

In [11]:
best = '''
        WITH
            total_purchase AS
                        (
                        SELECT 
                            customer_id,
                            SUM(total) total
                        FROM invoice
                        GROUP BY 1
                        )
        SELECT
            c.country,
            c.first_name || " " || c.last_name customer_name,
            MAX(tp.total) total_purchased
        FROM customer c
        INNER JOIN total_purchase tp ON tp.customer_id = c.customer_id
        GROUP BY 1
        ORDER BY 1;
'''
pd.read_sql_query(best, conn)

,country,customer_name,total_purchased
0,Argentina,Diego Gutiérrez,39.60
1,Australia,Mark Taylor,81.18
2,Austria,Astrid Gruber,69.30
3,Belgium,Daan Peeters,60.39
4,Brazil,Luís Gonçalves,108.90
5,Canada,François Tremblay,99.99
6,Chile,Luis Rojas,97.02
7,Czech Republic,František Wichterlová,144.54
8,Denmark,Kara Nielsen,37.62
9,Finland,Terhi Hämäläinen,79.20
